In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install jcopml

## Wine-quality-classifier 🍷🍾

Just follow this notebook till the end and you could predict quality of wine is good or bad.

**In this notebook we'll classify the quality of wine into Good or Bad.
Alltough in the data the target variable have numbers between 0-10 but we can classify it by classifying all obsevations as:**
        
* quality >= 7  as  good
* quality < 7 as bad

Let's firstly import data and analyse it 

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model
from jcopml.plot import plot_missing_value
from jcopml.feature_importance import mean_score_decrease

# Import Data & Feature Engineering

In [ ]:
df= pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df.head()

Let's check if there are any null values

In [ ]:
df.isna().sum()

Woow our data is not having any null value. So there's no need to clean the data. 😃😃

Now let's check the information about the features we have.

In [ ]:
df.info()

So we have total 1599 rows each having 12 features.
Let's explore the data 

In [ ]:
# Check Missing Value
plot_missing_value(df)

Dataset is Clean for NaN Values

In [ ]:
df['quality'].value_counts()

So we can see there are only 217 (199+18) records having quality >=7 i.e of good quality

Now let's change our target variable(quality) into good and bad.

In [ ]:
df['quality'] = ['Good' if x>=7 else 'bad' for x in df['quality']]
df.head()

See our quality variable have values good and bad.
Now our data needs just one thing more to be ready for modelling.

I used Pipeline Model for Fast Practice Build a Model

# Dataset Splitting

In [ ]:
X = df.drop(columns='quality')
y = df.quality

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Training Phase 1 : Simple Thinking

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from jcopml.tuning import random_search_params as rsp

In [ ]:
# Pipeline
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(), ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
                             'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
                             'pH', 'sulphates', 'alcohol']),
])

pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', XGBClassifier(n_jobs=-1, random_state=42))
])

# Training
model = RandomizedSearchCV(pipeline, rsp.xgb_params, cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

# Evaluation
print("Best Parameter for XGB is {}".format(model.best_params_))
print("Score for Data Train is {}".format(model.score(X_train, y_train)))
print("Best Score is {}".format(model.best_score_))
print("Score for Data Test is {}".format(model.score(X_test, y_test)))

# Phase 2 : Feature Importance

In [ ]:
df_imp = mean_score_decrease(X_train, y_train, model, plot=True)

I think all feature is importance to include in model... :)

# Phase 3 : Polynomial

In [ ]:
# Pipeline
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(poly=2), ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
                             'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
                             'pH', 'sulphates', 'alcohol']),
])

pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', XGBClassifier(n_jobs=-1, random_state=42))
])

# Training
model = RandomizedSearchCV(pipeline, rsp.xgb_poly_params, cv=3, n_jobs=-1, verbose=1)
model.fit(X_train, y_train)

# Evaluation
print("Best Parameter for XGB is {}".format(model.best_params_))
print("Score for Data Train is {}".format(model.score(X_train, y_train)))
print("Best Score is {}".format(model.best_score_))
print("Score for Data Test is {}".format(model.score(X_test, y_test)))

# Conclusion Model

In [ ]:
parameter = {'algo__colsample_bytree': 0.5163168100195784,
             'algo__gamma': 1,
             'algo__learning_rate': 0.24410494920734804,
             'algo__max_depth': 8,
             'algo__n_estimators': 172,
             'algo__reg_alpha': 0.015391401698302503,
             'algo__reg_lambda': 0.025101867367220122,
             'algo__subsample': 0.7658130344388201}

# Pipeline
preprocessor = ColumnTransformer([
    ('numeric', num_pipe(), ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
                             'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
                             'pH', 'sulphates', 'alcohol']),
])

pipeline = Pipeline([
    ('prep', preprocessor),
    ('algo', XGBClassifier(parameter=parameter, n_jobs=-1, random_state=42))
])

# Training
model = pipeline.fit(X_train, y_train)

# Evaluation
print("Score for Data Train is {}".format(model.score(X_train, y_train)))
print("Score for Data Test is {}".format(model.score(X_test, y_test)))

I found best Parameter and I fitting to model :)

**So we can see that Score Highest of XGB Classifier is 91%**

**Conclusion: After 3 Phase, I found Best Parameter and I Fiiting to Model and Get Score 91%**

**PLEASE UPVOTE MY NOTEBOOK IF IT HELPED YOU** 😊😊